In [7]:

import pandas as pd
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from google.cloud import bigquery
!pip install db-dtypes
!pip install google-cloud-bigquery
!pip install google-cloud-bigquery-storage


In [18]:
data

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,179.0,138.0,169.0,180.0,137.0,169.0,182.0,139.0,169.0,186.0,...,189.0,161.0,192.0,189.0,161.0,189.0,187.0,157.0,183.0,4.0
1,139.0,74.0,78.0,148.0,80.0,85.0,160.0,91.0,99.0,170.0,...,9.0,8.0,8.0,15.0,14.0,16.0,21.0,18.0,22.0,4.0
2,65.0,41.0,57.0,130.0,95.0,115.0,173.0,136.0,161.0,187.0,...,20.0,9.0,18.0,19.0,10.0,19.0,17.0,10.0,17.0,4.0
3,201.0,159.0,176.0,207.0,155.0,174.0,207.0,150.0,163.0,207.0,...,83.0,86.0,138.0,104.0,101.0,161.0,117.0,102.0,156.0,6.0
4,229.0,188.0,208.0,237.0,191.0,216.0,235.0,179.0,195.0,233.0,...,106.0,105.0,168.0,131.0,122.0,193.0,142.0,121.0,184.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10010,26.0,17.0,27.0,52.0,38.0,51.0,47.0,32.0,45.0,38.0,...,37.0,24.0,36.0,54.0,37.0,51.0,41.0,28.0,41.0,2.0
10011,183.0,119.0,145.0,213.0,146.0,177.0,222.0,146.0,181.0,224.0,...,227.0,126.0,173.0,228.0,122.0,160.0,224.0,96.0,118.0,1.0
10012,245.0,129.0,140.0,247.0,129.0,142.0,248.0,132.0,139.0,248.0,...,234.0,129.0,131.0,230.0,122.0,127.0,226.0,120.0,122.0,4.0
10013,253.0,168.0,182.0,254.0,171.0,185.0,254.0,164.0,180.0,254.0,...,247.0,175.0,167.0,244.0,171.0,163.0,243.0,172.0,163.0,4.0


In [19]:
metadata = pd.read_csv("HAM10000_metadata.csv")

metadata

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face


In [20]:
metadata = pd.read_csv("HAM10000_metadata.csv")
label_map = {'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}
metadata['label'] = metadata['dx'].map(label_map)
metadata

,lesion_id,image_id,dx,dx_type,age,sex,localization,label
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,2
...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,0
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,0
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,0
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,0


In [21]:

image_dir = "HAM10000_all_images" 
IMG_SIZE = 64

images = []
labels = []

for i, row in metadata.iterrows():
    img_path = os.path.join(image_dir, row['image_id'] + '.jpg')
    if os.path.exists(img_path):
        img = Image.open(img_path).resize((IMG_SIZE, IMG_SIZE))
        images.append(np.array(img) / 255.0)
        labels.append(row['label'])

X = np.array(images)
y = to_categorical(labels, num_classes=7)

print("Image shape:", X.shape)
print("Label shape:", y.shape)

Image shape: (0,)
Label shape: (0, 7)


In [22]:

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)





ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax') 
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', 'recall'])

model.summary()


In [ ]:
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=20,
                    batch_size=32)


In [ ]:

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('CNN Training')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
metadata['dx'].unique()


In [ ]:
metadata['dx'].value_counts()
